# ACECQA Services EDA

Goal: Exploratory analysis of approved education and care services across Australia to inform equitable, cost-effective policy. Focus on:
- Service quality distribution and trends
- Accessibility and coverage (urban vs rural, transport access)
- Operational trends (capacity, hours, approval dates)

Data: `Education-services-with-station-access_loc.csv` enriched with geometry and nearest stations.

Outputs: reproducible visuals and summary tables for PPT.


In [ ]:
import os
import sys
import json
import math
from pathlib import Path

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point

pd.set_option('display.max_columns', 200)
DATA_DIR = Path('data')
MAIN_CSV = DATA_DIR / 'Education-services-with-station-access_loc.csv'
assert MAIN_CSV.exists(), f'Missing dataset: {MAIN_CSV}'


In [ ]:
def parse_geometry(geom_str: str):
    if pd.isna(geom_str):
        return np.nan, np.nan
    try:
        s = str(geom_str).strip().lower().replace('c(', '').replace(')', '')
        lon_str, lat_str = s.split(',')
        return float(lon_str), float(lat_str)
    except Exception:
        return np.nan, np.nan

raw = pd.read_csv(MAIN_CSV, dtype=str)
print('Raw shape:', raw.shape)
raw.head(3)


In [ ]:
# Clean basic columns and derive features

df = raw.copy()
# Standardize column names
df.columns = (
    df.columns.str.strip()
    .str.replace(' ', '_')
    .str.replace('/', '_')
    .str.replace('-', '_')
)

# Parse numerical columns
for c in ['NumberOfApprovedPlaces', 'Postcode', 'DistanceToTrainStation_km', 'DistanceToBusStation_km']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# Dates
for c in ['ServiceApprovalGrantedDate', 'Last_Service_Approval_Transfer_Date']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=True)

# Geometry
if 'geometry' in df.columns:
    lon_lat = df['geometry'].apply(parse_geometry)
    df['Longitude'] = lon_lat.apply(lambda t: t[0])
    df['Latitude'] = lon_lat.apply(lambda t: t[1])

# Approval year
if 'ServiceApprovalGrantedDate' in df.columns:
    df['ApprovalYear'] = df['ServiceApprovalGrantedDate'].dt.year

# Service type flags already exist; create concise type label
service_type = 'ServiceType' if 'ServiceType' in df.columns else None
if service_type:
    df['ServiceTypeSimple'] = df[service_type].fillna('Unknown')

print('Cleaned shape:', df.shape)
df[['State','Suburb','ServiceTypeSimple','NumberOfApprovedPlaces','OverallRating']].head(5)


Cleaned shape: (17663, 107)


,State,Suburb,ServiceTypeSimple,NumberOfApprovedPlaces,OverallRating
0,QLD,WATERFORD WEST,Centre-Based Care,60.0,Meeting NQS
1,QLD,EAGLEBY,Centre-Based Care,74.0,Working Towards NQS
2,QLD,BROADBEACH WATERS,Centre-Based Care,75.0,Meeting NQS
3,QLD,BURLEIGH WATERS,Centre-Based Care,106.0,Meeting NQS
4,QLD,CARRARA,Centre-Based Care,92.0,Meeting NQS


In [ ]:
# Operating hour span in hours (annual schedule)

def to_minutes(t):
    if pd.isna(t) or str(t).strip().upper() in {'NA', ''}:
        return np.nan
    s = str(t)
    try:
        if len(s) == 5 and ':' in s:
            h, m = s.split(':')
        else:
            # handle e.g. '07 30'
            parts = s.replace(' ',':').split(':')
            h, m = parts[0], parts[1]
        return int(h) * 60 + int(m)
    except Exception:
        return np.nan

if {'Annual_Monday_Start_Time','Annual_Monday_End_Time'}.issubset(df.columns):
    start_cols = [c for c in df.columns if c.endswith('Start_Time') and c.startswith('Annual_')]
    end_cols = [c for c in df.columns if c.endswith('End_Time') and c.startswith('Annual_')]
    # compute daily minutes then mean across Mon-Fri
    daily_hours = []
    for sc, ec in zip(start_cols, end_cols):
        start_min = df[sc].apply(to_minutes)
        end_min = df[ec].apply(to_minutes)
        span = (end_min - start_min) / 60.0
        daily_hours.append(span)
    if daily_hours:
        df['DailyHours_mean'] = pd.concat(daily_hours, axis=1).mean(axis=1, skipna=True)

# Flags for offerings already present as Yes/No; convert to bool
for col in ['Long_Day_Care','Preschool_Kindergarten___Part_of_a_School','Preschool_Kindergarten___Stand_alone','Outside_school_Hours_Care___After_School','Outside_school_Hours_Care___Before_School','Outside_school_Hours_Care___Vacation_Care','Other']:
    if col in df.columns:
        df[col + '_bool'] = df[col].map({'Yes': True, 'No': False}).fillna(False)

print(df[['DailyHours_mean'] + [c for c in df.columns if c.endswith('_bool')][:4]].head(3))


In [ ]:
# Build simple urban/rural proxy using ABS SOS categories if available, else postcode density heuristic

# Placeholder: if a column 'SOS' exists with identifiers 0/1/2/3 (Major Urban/Other Urban/Bounded Locality/Rural Balance)
if 'SOS' in df.columns:
    def sos_to_urban(s):
        try:
            s_int = int(str(s).strip())
            return 'Urban' if s_int in (0,1) else 'Rural'
        except Exception:
            return np.nan
    df['UrbanRural'] = df['SOS'].apply(sos_to_urban)
else:
    # Heuristic: treat state capital postcodes and dense suburbs as Urban; else Rural
    urban_states = {
        'NSW': ['2'], 'VIC': ['3'], 'QLD': ['4'], 'SA': ['5'], 'WA': ['6'], 'TAS': ['7'], 'NT': ['0'], 'ACT': ['2']
    }
    def heuristic_urban(row):
        pc = row.get('Postcode') if isinstance(row, pd.Series) else row
        st = row.get('State') if isinstance(row, pd.Series) else None
        if pd.isna(pc) or pd.isna(st):
            return np.nan
        s = str(int(float(pc))) if not isinstance(pc, str) else pc
        s = s.strip()
        if st == 'ACT':
            return 'Urban'
        # inner metro: distance to bus/train < 2km or postcode starts with metro leading digit
        near_pt = False
        for col in ['DistanceToTrainStation_km','DistanceToBusStation_km']:
            if col in df.columns:
                try:
                    val = float(row[col])
                    if not math.isnan(val) and val <= 2.0:
                        near_pt = True
                except Exception:
                    pass
        if near_pt:
            return 'Urban'
        # fallback by leading postcode digit per state
        lead = s[0]
        return 'Urban' if lead in urban_states.get(st, []) else 'Rural'
    df['UrbanRural'] = df.apply(heuristic_urban, axis=1)

# Transport accessibility score (lower distance => higher score)
for col in ['DistanceToTrainStation_km','DistanceToBusStation_km']:
    if col in df.columns:
        df[col + '_clipped'] = df[col].clip(lower=0, upper=50)

if {'DistanceToTrainStation_km_clipped','DistanceToBusStation_km_clipped'}.issubset(df.columns):
    # Normalize to 0..1 where 1 is best accessibility
    dmax = 20.0
    train_score = 1 - (df['DistanceToTrainStation_km_clipped'] / dmax)
    bus_score = 1 - (df['DistanceToBusStation_km_clipped'] / dmax)
    df['PT_Access_Score'] = train_score.combine(bus_score, func=lambda a,b: np.nanmean([a,b]))

df[['UrbanRural','PT_Access_Score']].head(5)


In [ ]:
# Distribution by state and service type

state_counts = df.groupby('State').size().reset_index(name='Services')
fig1 = px.bar(state_counts, x='State', y='Services', title='Number of Services by State/Territory')
fig1.show()

if 'ServiceTypeSimple' in df.columns:
    type_counts = df.groupby(['State','ServiceTypeSimple']).size().reset_index(name='Count')
    fig2 = px.bar(type_counts, x='State', y='Count', color='ServiceTypeSimple', barmode='stack',
                  title='Service Types Distribution by State')
    fig2.show()

# Map of services (sample for performance)
sample = df.sample(n=min(8000, len(df)), random_state=42)
fig_map = px.scatter_geo(sample, lon='Longitude', lat='Latitude', color='ServiceTypeSimple',
                         hover_name='ServiceName', hover_data=['Suburb','State','OverallRating'],
                         opacity=0.6, title='Service Locations (sample)')
# Plotly geo does not support scope='australia'. Center and bound Australia manually.
fig_map.update_geos(scope='world', projection_type='natural earth',
                    center=dict(lat=-25, lon=134),
                    lataxis_range=[-44, -10], lonaxis_range=[112, 155])
fig_map.show()
import plotly.express as px


In [ ]:
# Ratings analysis

# Normalize OverallRating into ordered categories
rating_order = [
    'Significant Improvement Required','Working Towards NQS','Meeting NQS','Exceeding NQS','Excellent'
]
if 'OverallRating' in df.columns:
    df['OverallRating_cat'] = pd.Categorical(df['OverallRating'], categories=rating_order, ordered=True)

    rating_state = (
        df.groupby(['State','OverallRating_cat']).size().reset_index(name='Count')
    )
    fig_r1 = px.bar(rating_state, x='State', y='Count', color='OverallRating_cat', barmode='stack',
                    category_orders={'OverallRating_cat': rating_order},
                    title='Overall Ratings by State')
    fig_r1.show()

    if 'ServiceTypeSimple' in df.columns:
        rating_type = (
            df.groupby(['ServiceTypeSimple','OverallRating_cat']).size().reset_index(name='Count')
        )
        fig_r2 = px.bar(rating_type, x='ServiceTypeSimple', y='Count', color='OverallRating_cat', barmode='stack',
                        category_orders={'OverallRating_cat': rating_order},
                        title='Overall Ratings by Service Type')
        fig_r2.show()

    # Proximity vs rating: boxplots
    for dist_col in ['DistanceToTrainStation_km','DistanceToBusStation_km']:
        if dist_col in df.columns:
            fig = px.box(df, x='OverallRating_cat', y=dist_col, category_orders={'OverallRating_cat': rating_order},
                         title=f'{dist_col} by Overall Rating')
            fig.show()




In [ ]:
# Operational trends: capacity and hours by state and over time

# Capacity distribution by state
fig_c1 = px.box(df, x='State', y='NumberOfApprovedPlaces', points='outliers',
               title='Capacity (Approved Places) by State')
fig_c1.show()

# Capacity vs approval year trend
if 'ApprovalYear' in df.columns:
    cap_year = df.groupby('ApprovalYear')['NumberOfApprovedPlaces'].median().reset_index()
    fig_c2 = px.line(cap_year, x='ApprovalYear', y='NumberOfApprovedPlaces', title='Median Capacity over Years')
    fig_c2.show()

# Daily hours by service type
if 'DailyHours_mean' in df.columns and 'ServiceTypeSimple' in df.columns:
    fig_c3 = px.box(df, x='ServiceTypeSimple', y='DailyHours_mean', points='outliers',
                    title='Operating Hours by Service Type')
    fig_c3.show()

# Does transport accessibility relate to capacity or hours?
if 'PT_Access_Score' in df.columns:
    fig_c4 = px.scatter(df.sample(min(len(df), 8000), random_state=1),
                        x='PT_Access_Score', y='NumberOfApprovedPlaces', color='UrbanRural',
                        trendline='ols',
                        title='Capacity vs Public Transport Accessibility')
    fig_c4.show()
    if 'DailyHours_mean' in df.columns:
        fig_c5 = px.scatter(df.sample(min(len(df), 8000), random_state=2),
                            x='PT_Access_Score', y='DailyHours_mean', color='UrbanRural',
                            trendline='ols',
                            title='Operating Hours vs Public Transport Accessibility')
        fig_c5.show()
fig.write_html("Operating Hours vs Public Transport Accessibility.html")

In [ ]:
# Accessibility analysis

# Distance distributions by state and Urban/Rural
for dist_col in ['DistanceToTrainStation_km','DistanceToBusStation_km']:
    if dist_col in df.columns:
        fig = px.violin(df, x='State', y=dist_col, color='UrbanRural', box=True, points=False,
                        title=f'{dist_col} Distribution by State and Urban/Rural')
        fig.show()

# Identify underserved areas: services with distances > thresholds
THRESH_TRAIN = 10
THRESH_BUS = 5
conds = []
if 'DistanceToTrainStation_km' in df.columns:
    conds.append(df['DistanceToTrainStation_km'] > THRESH_TRAIN)
if 'DistanceToBusStation_km' in df.columns:
    conds.append(df['DistanceToBusStation_km'] > THRESH_BUS)
if conds:
    underserved = df[np.logical_or.reduce(conds)].copy()
    underserved['UnderservedFlag'] = True
    by_state = underserved.groupby(['State','UrbanRural']).size().reset_index(name='UnderservedServices')
    fig_u = px.bar(by_state, x='State', y='UnderservedServices', color='UrbanRural', barmode='group',
                   title='Potentially Underserved Services (far from PT)')
    fig_u.show()
    # Save table for PPT
    underserved_cols = ['ServiceApprovalNumber','ServiceName','State','Suburb','Postcode','UrbanRural',
                        'DistanceToTrainStation_km','DistanceToBusStation_km','NumberOfApprovedPlaces','OverallRating']
    underserved[underserved_cols].to_csv('underserved_services.csv', index=False)

# Map underserved services
if len(underserved) > 0 and {'Longitude','Latitude'}.issubset(underserved.columns):
    fig_umap = px.scatter_geo(underserved, lon='Longitude', lat='Latitude', color='UrbanRural',
                              hover_name='ServiceName', hover_data=['Suburb','State'],
                              title='Underserved Services Map')
    fig_umap.update_geos(scope='world', projection_type='natural earth',
                         center=dict(lat=-25, lon=134),
                         lataxis_range=[-44, -10], lonaxis_range=[112, 155])
    fig_umap.show()
fig.write_html("underserved.html")

In [ ]:
# Correlations between quality rating and features

from scipy.stats import spearmanr

# Encode rating as ordinal for correlation
rating_to_num = {r:i for i,r in enumerate(rating_order, start=1)}
if 'OverallRating' in df.columns:
    df['OverallRating_num'] = df['OverallRating'].map(rating_to_num)

corr_features = ['NumberOfApprovedPlaces','DailyHours_mean','PT_Access_Score',
                 'DistanceToTrainStation_km','DistanceToBusStation_km']

available = [c for c in corr_features if c in df.columns]

corr_rows = []
for c in available:
    valid = df[['OverallRating_num', c]].dropna()
    if len(valid) > 100:
        rho, p = spearmanr(valid['OverallRating_num'], valid[c])
        corr_rows.append({'feature': c, 'spearman_rho': rho, 'p_value': p, 'n': len(valid)})

corr_df = pd.DataFrame(corr_rows).sort_values('spearman_rho', ascending=False)
print(corr_df)

fig_corr = px.bar(corr_df, x='feature', y='spearman_rho', color='p_value',
                  title='Spearman correlation with Overall Rating (ordinal)')
fig_corr.show()


                     feature  spearman_rho       p_value      n
2            PT_Access_Score      0.055053  1.904509e-12  16336
3  DistanceToTrainStation_km     -0.032019  4.254733e-05  16336
4    DistanceToBusStation_km     -0.054292  3.816903e-12  16336
0     NumberOfApprovedPlaces     -0.063015  1.549990e-15  15979
1            DailyHours_mean     -0.131848  8.789114e-40   9938


In [ ]:
# Save cleaned dataset and some summary tables

OUT_DIR = Path('outputs')
OUT_DIR.mkdir(exist_ok=True)

clean_cols = ['ServiceApprovalNumber','ServiceName','ProviderLegalName','ServiceType','State','Suburb','Postcode',
              'NumberOfApprovedPlaces','ServiceApprovalGrantedDate','ApprovalYear','OverallRating','UrbanRural',
              'DistanceToTrainStation_km','DistanceToBusStation_km','PT_Access_Score','Longitude','Latitude',
              'DailyHours_mean']

(df[clean_cols].copy()).to_csv(OUT_DIR / 'services_clean.csv', index=False)

# Summary: services per state x type
if 'ServiceTypeSimple' in df.columns:
    summary_state_type = df.groupby(['State','ServiceTypeSimple']).size().reset_index(name='Count')
    summary_state_type.to_csv(OUT_DIR / 'summary_state_type.csv', index=False)

# Summary: rating shares per state
if 'OverallRating_cat' in df.columns:
    rating_state = df.groupby(['State','OverallRating_cat']).size().reset_index(name='Count')
    rating_state.to_csv(OUT_DIR / 'summary_rating_state.csv', index=False)

print('Saved outputs to', OUT_DIR.resolve())


Saved outputs to C:\Users\11520\Desktop\1\13461088486\outputs


In [ ]:
# Location features vs service type
if 'ServiceTypeSimple' in df.columns:
    for dist_col in ['DistanceToTrainStation_km','DistanceToBusStation_km']:
        if dist_col in df.columns:
            fig = px.box(df, x='ServiceTypeSimple', y=dist_col, points='outliers',
                         title=f'{dist_col} by Service Type')
            fig.show()



In [ ]:
# Utility: save selected figures to static images (requires kaleido)
try:
    import kaleido  # noqa: F401
    HAS_KALEIDO = True
except Exception:
    HAS_KALEIDO = False

FIG_OUT = OUT_DIR / 'figures'
FIG_OUT.mkdir(exist_ok=True)

if HAS_KALEIDO:
    # Rebuild a few key charts and write pngs
    fig1 = px.bar(df.groupby('State').size().reset_index(name='Services'), x='State', y='Services',
                  title='Number of Services by State/Territory')
    fig1.write_image(str(FIG_OUT / 'services_by_state.png'), scale=2)

    if 'ServiceTypeSimple' in df.columns:
        type_counts = df.groupby(['State','ServiceTypeSimple']).size().reset_index(name='Count')
        fig2 = px.bar(type_counts, x='State', y='Count', color='ServiceTypeSimple', barmode='stack',
                      title='Service Types Distribution by State')
        fig2.write_image(str(FIG_OUT / 'types_by_state.png'), scale=2)

    if 'OverallRating_cat' in df.columns:
        rating_state = df.groupby(['State','OverallRating_cat']).size().reset_index(name='Count')
        fig_r1 = px.bar(rating_state, x='State', y='Count', color='OverallRating_cat', barmode='stack',
                        category_orders={'OverallRating_cat': rating_order},
                        title='Overall Ratings by State')
        fig_r1.write_image(str(FIG_OUT / 'ratings_by_state.png'), scale=2)

print('Figure directory:', FIG_OUT.resolve(), '| kaleido installed:', HAS_KALEIDO)


Figure directory: C:\Users\11520\Desktop\1\13461088486\outputs\figures | kaleido installed: False


In [ ]:
def parse_geometry(geom_str: str):
    if pd.isna(geom_str):
        return np.nan, np.nan
    try:
        # Expect format: c(lon, lat)
        s = str(geom_str).strip().lower().replace('c(', '').replace(')', '')
        lon_str, lat_str = s.split(',')
        return float(lon_str), float(lat_str)
    except Exception:
        return np.nan, np.nan

raw = pd.read_csv(MAIN_CSV, dtype=str)
print('Raw shape:', raw.shape)
raw.head(2)


Raw shape: (17663, 103)


,ServiceApprovalNumber,Provider Approval Number,ServiceName,ProviderLegalName,ServiceType,ServiceAddress,Suburb,State,Postcode,Phone,Fax,Conditions on Approval,NumberOfApprovedPlaces,ServiceApprovalGrantedDate,QualityArea1Rating,QualityArea2Rating,QualityArea3Rating,QualityArea4Rating,QualityArea5Rating,QualityArea6Rating,QualityArea7Rating,OverallRating,RatingsIssued,PreviousQualityArea1Rating,PreviousQualityArea2Rating,PreviousQualityArea3Rating,PreviousQualityArea4Rating,PreviousQualityArea5Rating,PreviousQualityArea6Rating,PreviousQualityArea7Rating,PreviousOverallRating,PreviousRatingsIssued,Last Service Approval Transfer Date,Long Day Care,Preschool/Kindergarten - Part of a School,Preschool/Kindergarten - Stand alone,Outside school Hours Care - After School,Outside school Hours Care - Before School,Outside school Hours Care - Vacation Care,Other,Annual Monday Start Time,Annual Monday End Time,Annual Tuesday Start Time,Annual Tuesday End Time,Annual Wednesday Start Time,Annual Wednesday End Time,Annual Thursday Start Time,Annual Thursday End Time,Annual Friday Start Time,Annual Friday End Time,Annual Saturday Start Time,Annual Saturday End Time,Annual Sunday Start Time,Annual Sunday End Time,School Terms Only Session 1 Monday Start Time,School Terms Only Session 1 Monday End Time,School Terms Only Session 1 Tuesday Start Time,School Terms Only Session 1 Tuesday End Time,School Terms Only Session1 Wednesday Start Time,School Terms Only Session1 Wednesday End Time,School Terms Only Session1 Thursday Start Time,School Terms Only Session1 Thursday End Time,School Terms Only Session 1 Friday Start Time,School Terms Only Session 1 Friday End Time,School Terms Only Session 1 Saturday Start Time,School Terms Only Session 1 Saturday End Time,School Terms Only Session1 Sunday Start Time,School Terms Only Session 1 Sunday End Time,School Terms Only Session 2 Monday Start Time,School Terms Only Session 2 Monday End Time,School Terms Only Session 2 Tuesday Start Time,School Terms Only Session 2 Tuesday End Time,School Terms Only Session 2 Wednesday Start Time,School Terms Only Session 2 Wednesday End Time,School Terms Only Session 2 Thursday Start Time,School Terms Only Session2 Thursday End Time,School Terms Only Session 2 Friday Start Time,School Terms Only Session 2 Friday End Time,School Terms Only Session 2 Saturday Start Time,School Terms Only Session 2 Saturday End Time,School Terms Only Session 2 Sunday Start Time,School Terms Only Session 2 Sunday End Time,Holiday Care Monday Start Time,Holiday Care Monday End Time,Holiday Care Tuesday Start Time,Holiday Care Tuesday End Time,Holiday Care Wednesday Start Time,Holiday Care Wednesday End Time,Holiday Care Thursday Start Time,Holiday Care Thursday End Time,Holiday Care Friday Start Time,Holiday Care Friday End Time,Holiday Care Saturday Start Time,Holiday Care Saturday End Time,Holiday Care Sunday Start Time,Holiday Care Sunday End Time,Temporarily Closed,FullAddress,geometry,ClosestTrainStation,ClosestBusStation,DistanceToTrainStation_km,DistanceToBusStation_km
0,SE-00000002,PR-00000898,Community Kids Waterford Early Education Centre,G8 Education Limited,Centre-Based Care,8-10 Allora St,WATERFORD WEST,QLD,4133,0732009852,NaN,This service is approved to provide education ...,60,03/11/2010,Meeting NQS,Meeting NQS,Meeting NQS,Meeting NQS,Meeting NQS,Meeting NQS,Meeting NQS,Meeting NQS,Sep 2023,Exceeding NQS,Meeting NQS,Exceeding NQS,Meeting NQS,Exceeding NQS,Exceeding NQS,Meeting NQS,Exceeding NQS,Sep 2016,NaN,Yes,No,Yes,No,No,No,No,06:15:00,18:15:00,06:15:00,18:15:00,06:15:00,18:15:00,06:15:00,18:15:00,06:15:00,18:15:00,NaN,NaN,NaN,NaN,15:00,18:15,15:00,18:15,15:00,18:15,15:00,18:15,15:00,18:15,NaN,NaN,NaN,NaN,06:15:00,08:30:00,06:15:00,08:30:00,06:15:00,08:30:00,06:15:00,08:30:00,06:15:00,08:30:00,NaN,NaN,NaN,NaN,06:15:00,18:15:00,06:15:00,18:15:00,06:15:00,18:15:00,06:15:00,18:15:00,06:15:00,18:15:00,NaN,NaN,NaN,NaN,No,"8-10 Allora St, WATERFORD WEST, QLD, 4133","c(153.13390291